# exp005_opt

In [1]:
import os
import sys
import itertools
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from optuna.integration import lightgbm as optuna_lgb
import line_notify

In [2]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [3]:
SEED = 42

ディレクトリ設定

In [4]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'exp005'
os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

データ読み込み

In [5]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

# データセット作成（レコメンド→対象データセット作成→特徴量エンジニアリング）

In [6]:
@noglobal
def get_customer_frequent(history, n=12, timedelta=None):
    """顧客ごと商品の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        dataframe: 抽出結果
    """
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()
        
    customer_agg = history.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index()
    customer_agg = customer_agg.rename(columns={'t_dat':'cnt'})
    customer_agg = customer_agg.sort_values(['customer_id', 'cnt'], ascending=False)
    result = customer_agg.groupby('customer_id').head(n)
    return result[['customer_id', 'article_id']]

@noglobal
def get_popular_article(history, n=12, timedelta=None):
    """全体の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        list: 抽出結果
    """
    # 全体の購入数量
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    total_agg = history.groupby('article_id')['t_dat'].count().reset_index()
    total_agg = total_agg.rename(columns={'t_dat':'cnt'})
    total_agg = total_agg.sort_values(['cnt'], ascending=False)
    total_agg = total_agg.head(n)
    result = list(total_agg['article_id'].values)
    return result

@noglobal
def get_reccomend(target_customer_id, history):
    """対象のcustomer_idに対するレコメンド結果を返す

    Args:
        target_customer_id (list): 対象のcustomer_id
        history (dataframe): レコメンドに用いる実績データ

    Returns:
        dataframe: レコメンド結果
    """
    n = 12
    result = pd.DataFrame()
    
    for td in [None, relativedelta(weeks=1), relativedelta(months=1), relativedelta(years=1)]:

        customer_freq = get_customer_frequent(history, n, td)
        popular_article = get_popular_article(history, n, td)
        # customerとpopular articleの全組み合わせでdataframe作成
        popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
        tmp = pd.concat([customer_freq, popular_article])
        result = result.append(tmp)
    result = result.drop_duplicates()
    result = result[result['customer_id'].isin(target_customer_id)].copy()

    return result

In [7]:
@noglobal
def add_labels(recom_result, history):
    """レコメンドしたデータが学習期間で購入されたかどうかのフラグを付与する

    Args:
        recom_result (_type_): レコメンド結果
        train_tran (_type_): 学習期間のトランザクションデータ

    Returns:
        _type_: 学習期間での購入フラグを付与したレコメンド結果
    """
    history = history[['customer_id', 'article_id']].drop_duplicates()
    history['buy'] = 1
    recom_result = recom_result.merge(history, on=['customer_id', 'article_id'], how='left')
    recom_result['buy'] = recom_result['buy'].fillna(0)
    return recom_result


In [8]:
@noglobal
def make_article_features(articles):
    le_cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
               'colour_group_name', 'perceived_colour_value_name', 'department_name',
               'index_name', 'index_group_name', 'section_name', 'garment_group_name']
    for c in le_cols:
        le = LabelEncoder()
        articles[c] = le.fit_transform(articles[c].fillna(''))
    return articles[['article_id']+le_cols]

@noglobal
def make_customer_features(customers):
    le_cols = ['club_member_status', 'fashion_news_frequency', 'postal_code']
    for c in le_cols:
        le = LabelEncoder()
        customers[c] = le.fit_transform(customers[c].fillna(''))
    return customers

@noglobal
def make_customer_article_features(target, history):
    df = target.merge(history, on=['customer_id', 'article_id'], how='inner')
    df = df.groupby(['customer_id', 'article_id']).agg({'t_dat':['count', 'min', 'max']}).reset_index()
    df.columns = ['customer_id', 'article_id', 'count', '1st_buy_date_diff', 'latest_buy_date_diff']
    df['1st_buy_date_diff'] = (history['t_dat'].max() - df['1st_buy_date_diff']).dt.days
    df['latest_buy_date_diff'] = (history['t_dat'].max() - df['latest_buy_date_diff']).dt.days
    return df

@noglobal
def add_features(df, history, articles, customers):
    article_features = make_article_features(articles)
    customer_features = make_customer_features(customers)
    customer_article_features = make_customer_article_features(df[['customer_id', 'article_id']], history)

    df = df.merge(article_features, on=['article_id'], how='left')
    df = df.merge(customer_features, on=['customer_id'], how='left')
    df = df.merge(customer_article_features, on=['article_id', 'customer_id'], how='left')
    return df
    

In [9]:
train_start = '2020-09-09'
valid_start = '2020-09-16'

In [10]:
# 学習データの作成
history_tran = transactions[transactions['t_dat'] < train_start].copy()
target_tran = transactions[(transactions['t_dat'] >= train_start) & (transactions['t_dat'] < valid_start)].copy()

target_id = target_tran['customer_id'].unique().tolist()
recom = get_reccomend(target_id, history_tran)
ml_train = add_labels(recom, target_tran)
ml_train = add_features(ml_train, history_tran, articles, customers)

In [11]:
# 評価データの作成
history_tran = transactions[transactions['t_dat'] < valid_start].copy()
target_tran = transactions[transactions['t_dat'] >= valid_start].copy()

target_id = target_tran['customer_id'].unique().tolist()
recom = get_reccomend(target_id, history_tran)
ml_valid = add_labels(recom, target_tran)
ml_valid = add_features(ml_valid, history_tran, articles, customers)

In [12]:
# テストデータの作成
target_id = sample['customer_id'].tolist()
recom = get_reccomend(target_id, transactions)
ml_test = add_features(recom, transactions, articles, customers)

In [13]:
ml_train['buy'].value_counts()

0.0    2977593
1.0      12998
Name: buy, dtype: int64

In [14]:
ml_valid['buy'].value_counts()

0.0    3254466
1.0      14544
Name: buy, dtype: int64

# レコメンド商品を購入するかどうかの2値分類モデル

In [15]:
target = 'buy'
not_use_cols = ['customer_id', 'article_id', target]
features = [c for c in ml_train.columns if c not in not_use_cols]

In [16]:
params = {
    "objective": "binary",
    "boosting" : "gbdt",
    "learning_rate": 0.1,
    "metric": "binary_logloss",
    "seed": SEED
}

In [17]:
tr_x, tr_y = ml_train[features], ml_train[target]
vl_x, vl_y = ml_valid[features], ml_valid[target]

tr_data = lgb.Dataset(tr_x, label=tr_y)
vl_data = lgb.Dataset(vl_x, label=vl_y)

model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                        num_boost_round=20000, early_stopping_rounds=100,verbose_eval=1000)

[I 2022-02-16 15:37:42,732] A new study created in memory with name: no-name-0fd4a6cf-a123-43bf-ac87-85eeee4062fb
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 12998, number of negative: 2977593
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.097426 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1376
[LightGBM] [Info] Number of data points in the train set: 2990591, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004346 -> initscore=-5.434075
[LightGBM] [Info] Start training from score -5.434075
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.025479:  14%|#4        | 1/7 [00:14<01:24, 14.08s/it]

Early stopping, best iteration is:
[26]	valid_0's binary_logloss: 0.0238092	valid_1's binary_logloss: 0.0254794


/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

Early stopping, best iteration is:
[477]	valid_0's binary_logloss: 0.0237456	valid_1's binary_logloss: 0.0252761


/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 12998, number of negative: 2977593
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.242883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1376
[LightGBM] [Info] Number of data points in the train set: 2990591, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004346 -> initscore=-5.434075
[LightGBM] [Info] Start training from score -5.434075
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's binary_logloss: 0.0234389	valid_1's binary_logloss: 0.0250832


min_data_in_leaf, val_score: 0.025048: 100%|##########| 5/5 [04:41<00:00, 56.28s/it]

Early stopping, best iteration is:
[1259]	valid_0's binary_logloss: 0.0233257	valid_1's binary_logloss: 0.0250521


In [18]:
model.params

{'objective': 'binary',
 'boosting': 'gbdt',
 'learning_rate': 0.1,
 'metric': 'binary_logloss',
 'seed': 42,
 'feature_pre_filter': False,
 'lambda_l1': 9.469353754507788e-08,
 'lambda_l2': 0.0022280155140637654,
 'num_leaves': 4,
 'feature_fraction': 0.41600000000000004,
 'bagging_fraction': 0.8154283192468761,
 'bagging_freq': 1,
 'min_child_samples': 50,
 'num_iterations': 20000,
 'early_stopping_round': 100}